In [4]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import numpy as np
from hilbert import decode, encode
from tqdm import tqdm
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import os

In [8]:
import gc


In [9]:
df = pd.read_parquet("data/F_BTCUSDT_1m.parquet")
tc = ["Open",
"High",
"Low",
"Close",
"Volume"]
for i in tc:
    df[i]=df[i].astype("float")

In [10]:
df1 = df.copy(deep=True)

In [11]:
ohlc_m,ohlc_std = pd.concat([df.Open,df.Close,df.High,df.Low]).mean(),pd.concat([df.Open,df.Close,df.High,df.Low]).std()

In [12]:
vol_m,vol_std = df.Volume.mean(),df.Volume.std()

In [13]:
ohlc_m,ohlc_std,vol_m,vol_std

(27993.054512542625, 17832.786089817815, 248.04630290656564, 426.867918968892)

In [14]:
df["Open"] = (df["Open"]-ohlc_m)/ohlc_std
df["High"] = (df["High"]-ohlc_m)/ohlc_std
df["Close"] = (df["Close"]-ohlc_m)/ohlc_std
df["Low"] = (df["Low"]-ohlc_m)/ohlc_std
df["Volume"] = (df["Volume"]-vol_m)/vol_std

In [15]:
rng = list(range(32*32,len(df),20))

In [16]:
div = int(len(rng)*0.9)

In [19]:
div

71234

In [17]:
70543 78381

70543

In [21]:
data = np.array(range(8*8))
locs = decode(data, 2, 3)
hilbert_list = []
target_list = []
target_list_real = []
cnt = 0
div = int(len(rng)*0.9)

for i in tqdm(range(32*32,len(df),20)):
    cnt+=1
    temp_f = df1.iloc[i-5:i]
    temp_t_5 = df1.iloc[i:i+5]
    temp_t_10 = df1.iloc[i+5:i+10]
    temp_t_15 = df1.iloc[i+10:i+15]
    temp_t_20 = df1.iloc[i+15:i+20]
    
    #if len(temp_t_15)<5:
    #    target_list.append(
    #    np.array([0,0,0,0,0,0,0,0])
    #)
    #    continue
    #if i>64*64+1000:
    #    break
    lv = temp_f.Close.iloc[-1]
    o1 = np.zeros((5,8,8))
    vals_f = df.iloc[i-8*8:i][["Open","High","Low","Close","Volume"]].values
    
    #vals = temp_t_15[["Open","High","Low","Close","Volume"]].values
    #target_list_real.append(temp_t_15[["Open","High","Low","Close"]].values)
    #vals[:,:4]=(vals[:,:4]-vals[:,2].min())/(vals[:,1].max()-vals[:,2].min())
    #vals[:,4]=(vals[:,4]-vals[:,4].min())/(vals[:,4].max()-vals[:,4].min())
    for j in range(len(data)):
        o1[:,locs[j][0],locs[j][1]] = vals_f[j]
    if cnt<div:
        np.save('imgs/train_4layer_w_volume/s8_%07d.npy'%cnt,o1)
    else:
        np.save('imgs/test_4layer_w_volume/s8_%07d.npy'%cnt,o1)

    #hilbert_list.append(o1)
    del o1
    del vals_f
    del temp_f
    if not cnt%1000: 
        gc.get_threshold()
        gc.collect()
    #target_list.append(vals)
    
    """ target_list.append(
        np.array(
            [
                (temp_t_5.Low.min()/lv)-1,(temp_t_5.High.max()/lv)-1,
                (temp_t_10.Low.min()/lv)-1,(temp_t_10.High.max()/lv)-1,
                (temp_t_15.Low.min()/lv)-1,(temp_t_15.High.max()/lv)-1,
                (temp_t_20.Low.min()/lv)-1,(temp_t_20.High.max()/lv)-1
            ]
        )
    )
    """

100%|███████████████████████████████████| 79149/79149 [00:51<00:00, 1525.80it/s]


In [9]:
hlt = np.array(hilbert_list)

In [25]:
ohlc = hlt[:,:4,:,:]
vol = hlt[:,-1:,:,:]
print(ohlc.mean(),ohlc.std(),vol.mean(),vol.std())

8254.983861797069 78.04553313546148 23.977786002281874 40.07729618887132


In [32]:
g1 = ohlc[:50]
g2 = ohlc[50:-1]
n=len(g1)
m=len(g2)
xm=g1.mean()
ym=g2.mean()
xs=g1.std()
ys=g2.std()

In [33]:
(xm*n+ym*m)/(m+n)

8255.268965075682

In [27]:

q1 = (len(g1)-1)*g1.var() + len(g1)*g1.mean()**2; q1
q2 = (len(g2)-1)*g2.var() + len(g2)*g2.mean()**2; q2
qc = q1 + q2
sc = np.sqrt( (qc - (len(g1)+len(g2))*ohlc.mean()**2)/(len(g1)+len(g2)-1) ); sc

77.67980850162729

In [30]:
np.sqrt((((n-1)*xs**2+(m-1)*ys**2)/(n+m-1)) + ((n*m*(xm-ym)**2)/((n+m)*(n+m-1))))

77.67980850187215

In [ ]:
q1 = (n1-1)*var(x1) + n1*mean(x1)^2; q1
[1] 408219.2 
q2 = (n2-1)*var(x2) + n2*mean(x2)^2; q1
[1] 123819.4
qc = q1 + q2
sc = sqrt( (qc - (n1+n2)*mean(x)^2)/(n1+n2-1) ); sc

In [4]:
target_list_real[-1]=target_list_real[-2]

In [5]:
target_list = [i.flatten() for i in target_list]

In [6]:
target_list[-1]=target_list[-4]
target_list[-2]=target_list[-4]
target_list[-3]=target_list[-4]

In [7]:
target_list = [i.reshape(1,*i.shape) for i in target_list]

In [8]:
tla = np.concatenate(target_list,axis=0)

In [9]:
tla.shape

(158298, 150)

In [28]:
target_df = pd.DataFrame(target_list,columns=["Low_5","High_5","Low_10","High_10","Low_15","High_15","Low_20","High_20"])

In [87]:
target_df = pd.DataFrame(kmeans,columns=["cluster_label"])

In [89]:
target_df  =pd.get_dummies(target_df.cluster_label)

In [6]:
ohlc.mean(),ohlc.std(),vol.mean(),vol.std()

(28001.938870730588, 17834.47169281324, 248.00923831004107, 426.5438396373735)

In [91]:
target_df.columns = ["cluster_"+str(i) for i in target_df.columns]

In [ ]:
hlt = np.array(hilbert_list)
#hlt = (hlt-hlt.mean())/hlt.std()
ohlc = hlt[:,:4,:,:]
vol = hlt[:,-1:,:,:]
print(ohlc.mean(),ohlc.std(),vol.mean(),vol.std())
ohlc = (ohlc-ohlc.mean())/ohlc.std()
vol = (vol-vol.mean())/vol.std()
hltn = np.concatenate((ohlc,vol),axis=1)

div = int(len(target_list)*0.9)
for i,j in tqdm(enumerate(hltn)):
    if i<div:
        np.save('imgs/train_4layer_w_volume/%07d.npy'%i,j)
    else:
        np.save('imgs/test_4layer_w_volume/%07d.npy'%i,j) 
#target_df["label"] = target_df.index.to_list()
#target_df["label"] = target_df.label.apply(lambda x:"%07d.npy"%x)
#target_df.iloc[:div].to_parquet("imgs/train_4layer_w_volume/target_cluster_1D.parquet")
#target_df.iloc[div:].to_parquet("imgs/test_4layer_w_volume/target_cluster_1D.parquet")

In [47]:
target_df

,Low_5,High_5,Low_10,High_10,Low_15,High_15,Low_20,High_20
0,-0.001045,0.000628,-0.001065,-0.000290,-0.000995,0.000169,-0.000500,0.000348
1,-0.000710,0.000480,-0.000813,0.000588,-0.000168,0.001066,-0.000080,0.001144
2,-0.000391,0.001111,-0.001286,0.000123,-0.001916,-0.000412,-0.001526,-0.000255
3,-0.000938,0.000065,-0.001433,-0.000519,-0.001558,-0.000644,-0.002621,-0.001061
4,-0.000211,0.000055,-0.000857,-0.000134,-0.000984,0.000109,-0.000310,0.000408
...,...,...,...,...,...,...,...,...
78376,-0.000120,0.002944,0.000951,0.002197,0.000339,0.002008,0.000523,0.001664
78377,-0.000627,0.000249,-0.000030,0.006984,0.004845,0.008427,0.003855,0.006188
78378,-0.002091,0.000248,-0.001878,0.000213,-0.001154,0.001506,0.000337,0.003518
78379,-0.001145,0.000721,-0.000336,0.007139,0.002088,0.005362,0.002503,0.005223


In [31]:
div

71233

In [30]:
div=div-1
target_df["label"] = target_df.index.to_list()
target_df["label"] = target_df.label.apply(lambda x:"%07d.npy"%(x+1))
target_df.iloc[:div].to_parquet("imgs/train_4layer_w_volume/target_next_20.parquet")
target_df.iloc[div:].to_parquet("imgs/test_4layer_w_volume/target_next_20.parquet")

In [49]:
pd.read_parquet("imgs/train_4_layer_w_volume_128/target_next_20_128.parquet")

,Low_5,High_5,Low_10,High_10,Low_15,High_15,Low_20,High_20,label
0,-0.001045,0.000628,-0.001065,-0.000290,-0.000995,0.000169,-0.000500,0.000348,0000001.npy
1,-0.000710,0.000480,-0.000813,0.000588,-0.000168,0.001066,-0.000080,0.001144,0000002.npy
2,-0.000391,0.001111,-0.001286,0.000123,-0.001916,-0.000412,-0.001526,-0.000255,0000003.npy
3,-0.000938,0.000065,-0.001433,-0.000519,-0.001558,-0.000644,-0.002621,-0.001061,0000004.npy
4,-0.000211,0.000055,-0.000857,-0.000134,-0.000984,0.000109,-0.000310,0.000408,0000005.npy
...,...,...,...,...,...,...,...,...,...
70537,-0.002785,0.000024,-0.003862,0.000348,-0.003128,0.000269,-0.001136,0.000250,0070538.npy
70538,-0.002545,0.001493,0.000450,0.002291,0.000553,0.002076,-0.000651,0.001875,0070539.npy
70539,-0.000806,0.000279,-0.000806,0.000420,-0.000733,0.001774,0.001256,0.002620,0070540.npy
70540,-0.000083,0.001868,-0.000678,0.001810,-0.001307,0.000771,-0.000541,0.000605,0070541.npy


In [50]:
pd.read_parquet("imgs/test_4_layer_w_volume_128/target_next_20_128.parquet")

,Low_5,High_5,Low_10,High_10,Low_15,High_15,Low_20,High_20,label
70542,-0.000713,0.001835,0.000849,0.008640,0.003803,0.007507,0.006019,0.008616,0070543.npy
70543,-0.001004,0.005147,0.002207,0.007466,0.001751,0.006942,-0.000485,0.003245,0070544.npy
70544,-0.002585,0.001392,-0.002338,-0.000388,-0.003793,-0.001111,-0.003759,-0.001154,0070545.npy
70545,-0.000170,0.001357,-0.001600,0.001269,-0.002791,-0.000836,-0.005698,-0.002543,0070546.npy
70546,-0.001468,0.000005,-0.005545,-0.000473,-0.006164,-0.003784,-0.006895,-0.004452,0070547.npy
...,...,...,...,...,...,...,...,...,...
78376,-0.000120,0.002944,0.000951,0.002197,0.000339,0.002008,0.000523,0.001664,0078377.npy
78377,-0.000627,0.000249,-0.000030,0.006984,0.004845,0.008427,0.003855,0.006188,0078378.npy
78378,-0.002091,0.000248,-0.001878,0.000213,-0.001154,0.001506,0.000337,0.003518,0078379.npy
78379,-0.001145,0.000721,-0.000336,0.007139,0.002088,0.005362,0.002503,0.005223,0078380.npy


In [42]:
tla = np.nan_to_num(tla)

In [85]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=16, random_state=0).fit_predict(tla)


In [38]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=10, random_state=0).fit_predict(target_df.iloc[:,:-2].values)


In [95]:
np.unique(kmeans,return_counts=True)[1]/158297

array([0.06228798, 0.06561085, 0.05672881, 0.05523794, 0.07700714,
       0.05455568, 0.05410715, 0.04962191, 0.05382919, 0.0595463 ,
       0.07561103, 0.05974845, 0.05586335, 0.08029843, 0.05971686,
       0.08023525])

In [86]:
dfc = pd.DataFrame(tla)
dfc["label"] = kmeans

In [73]:
dsc = dfc.groupby("label").describe()

In [85]:
dfc

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,label
0,0.884841,0.909735,0.865209,0.888079,0.208722,0.885246,1.000000,0.883020,0.910949,0.290546,0.909937,0.911354,0.751265,0.751265,0.226090,0.770694,0.770694,0.596843,0.645618,0.270613,0.658166,0.658166,0.459016,0.459016,0.231354,0.454564,0.471362,0.269379,0.294880,0.734727,0.309856,0.380692,0.304392,0.318559,0.834280,0.326452,0.384538,0.326250,0.364096,0.180846,0.362477,0.416920,0.221615,0.280105,0.309454,0.271807,0.271807,0.158672,0.186400,0.399544,0.190447,0.282736,0.162720,0.163732,0.054233,0.176280,0.177899,0.016798,0.052621,0.619705,0.050395,0.193078,0.000000,0.131350,0.718100,0.167780,0.206436,0.136410,0.206436,0.059382,0.183161,0.280308,0.183161,0.280308,0.252599,0.282129,0.380085,0.242866,0.376847,0.148713,0.377454,0.526209,0.370775,0.526209,0.219591,0.542400,0.542400,0.399919,0.425218,0.216988,0.425015,0.586926,0.333536,0.586723,0.916181,0.586926,0.688120,0.586926,0.622546,0.830594,0.668691,0.759765,0.620725,0.647237,0.528532,0.621129,0.647845,0.383323,0.383323,0.646005,0.445456,0.445456,0.172232,0.263307,0.229910,0.258854,0.379680,0.059705,0.201983,1.000000,0.201781,0.263914,0.065371,0.222627,0.359753,0.222627,0.390002,0.220401,0.252176,0.478043,0.252985,0.317749,0.203400,0.222627,0.117872,0.317749,0.440397,0.236592,0.422789,0.000000,0.363489,0.586926,0.363489,0.586926,0.127829,0.584092,0.696013,0.522364,0.629630,0.239829,2
1,0.198649,0.294912,0.169728,0.170783,0.098319,0.183872,0.185560,0.017522,0.054887,0.637433,0.052565,0.201393,0.000000,0.137007,0.731240,0.175005,0.215326,0.142284,0.215326,0.103228,0.191049,0.292379,0.191049,0.292379,0.287438,0.294279,0.396453,0.253325,0.393076,0.188395,0.393709,0.548871,0.386743,0.548871,0.255969,0.565759,0.565759,0.417142,0.443530,0.253487,0.443319,0.612202,0.347900,0.611991,0.920088,0.612202,0.717754,0.612202,0.649356,0.838491,0.697488,0.792485,0.647456,0.675111,0.550509,0.647878,0.675744,0.399831,0.399831,0.662506,0.464640,0.464640,0.179650,0.274646,0.265807,0.270002,0.396031,0.062276,0.210682,1.000000,0.210471,0.275280,0.068187,0.232214,0.389597,0.232214,0.406798,0.229892,0.263036,0.502373,0.263880,0.331433,0.212160,0.232214,0.158991,0.331433,0.459362,0.246781,0.440996,0.046614,0.379143,0.612202,0.379143,0.612202,0.168484,0.609246,0.725987,0.544860,0.656745,0.275264,0.703399,0.776441,0.565337,0.734220,0.618428,0.734642,1.000000,0.726198,0.859194,0.443621,0.823306,0.823728,0.501794,0.612202,0.797728,0.611146,0.755541,0.530082,0.713743,0.116363,0.708254,0.708254,0.530082,0.651467,0.063263,0.530293,0.740764,0.529449,0.733798,0.041306,0.687566,0.687777,0.612413,0.612413,0.000000,0.615157,0.670044,0.454085,0.568503,0.247437,0.567237,0.567237,0.385898,0.385898,0.159064,0.389909,0.401098,0.241081,0.269791,0.201312,10
2,0.684153,0.783337,0.631915,0.660789,0.550509,0.632356,0.661450,0.373374,0.373374,0.662506,0.441040,0.441040,0.143487,0.242671,0.265807,0.237822,0.369407,0.020939,0.175887,1.000000,0.175667,0.243333,0.027110,0.198369,0.389597,0.198369,0.380648,0.195944,0.230549,0.502373,0.231430,0.301962,0.177430,0.198369,0.158991,0.301962,0.435530,0.213577,0.416354,0.046614,0.351774,0.595107,0.351774,0.595107,0.168484,0.592021,0.713908,0.524796,0.641613,0.275264,0.690324,0.766586,0.546176,0.722504,0.618428,0.722945,1.000000,0.714128,0.852987,0.443621,0.815517,0.815958,0.479832,0.595107,0.797728,0.594005,0.744765,0.509367,0.701124,0.116363,0.695393,0.695393,0.509367,0.636103,0.063263,0.509588,0.729337,0.508706,0.722063,0.041306,0.673793,0.674014,0.595327,0.595327,0.000000,0.598193,0.655499,0.430020,

In [74]:
col_list = []
for i,j in enumerate(dsc.columns):
    if j[0]%5==0:
        val="Open"
    if j[0]%5==1:
        val="High"
    if j[0]%5==2:
        val="Low"
    if j[0]%5==3:
        val="Close"
    if j[0]%5==4:
        val="Volume"
    col_list.append(val+"_"+str((j[0]//5)+1)+"_"+j[1]) 

In [75]:
dsc.columns=col_list

In [101]:
dsc.to_parquet("20_cluster_desc.parquet")

In [80]:
target_df["labels"]=kmeans

In [84]:
target_df

,cluster_label,label
0,0,0000000.npy
1,1,0000001.npy
2,1,0000002.npy
3,0,0000003.npy
4,0,0000004.npy
...,...,...
158293,1,0158293.npy
158294,1,0158294.npy
158295,1,0158295.npy
158296,1,0158296.npy


In [84]:
dsc = pd.read_parquet("20_cluster_desc.parquet")

In [153]:
np.array([ta[[j+"_"+str(i)+"_mean" for i in range(1,16)]].to_list() for j in ["Open","High","Low","Close"]]).transpose()

array([[1.00000014, 1.00050549, 0.99949736, 0.99999538],
       [0.99999523, 1.0005439 , 0.99945056, 0.99998758],
       [0.99998758, 1.00049886, 0.99948234, 0.99999244],
       [0.99999257, 1.00049624, 0.99949488, 0.999985  ],
       [0.99998534, 1.00047494, 0.99949247, 0.99997734],
       [0.99997688, 1.00046857, 0.99949614, 0.99997232],
       [0.99997212, 1.00053876, 0.9994444 , 0.99999348],
       [0.99999371, 1.00049089, 0.99946937, 0.99996677],
       [0.99996704, 1.00046886, 0.99943501, 0.99993834],
       [0.99993799, 1.00054706, 0.9992615 , 0.99987931],
       [0.99988027, 1.00103297, 0.99867024, 0.99990395],
       [0.99990301, 1.00071938, 0.99915947, 0.99995427],
       [0.99995398, 1.00057096, 0.99931628, 0.99993812],
       [0.99993745, 1.00050631, 0.99934598, 0.9999205 ],
       [0.99992095, 1.00047462, 0.99936369, 0.99990515]])

In [209]:
target_list_real[46]

array([[8312.87, 8314.38, 8308.51, 8310.  ],
       [8306.86, 8310.91, 8306.86, 8308.02],
       [8310.24, 8315.68, 8306.86, 8312.32],
       [8312.22, 8312.48, 8309.09, 8309.09],
       [8311.62, 8311.62, 8308.28, 8309.47],
       [8309.39, 8309.76, 8306.86, 8306.86],
       [8308.92, 8308.92, 8303.1 , 8304.64],
       [8304.64, 8308.  , 8276.37, 8279.02],
       [8276.35, 8290.53, 8275.81, 8283.14],
       [8281.51, 8285.56, 8279.73, 8282.78],
       [8282.53, 8284.75, 8280.38, 8281.05],
       [8281.21, 8287.36, 8279.32, 8286.7 ],
       [8287.57, 8292.07, 8286.79, 8292.07],
       [8290.96, 8294.93, 8289.08, 8292.68],
       [8289.08, 8291.02, 8285.23, 8287.82]])

In [19]:
target_df

NameError: name 'target_df' is not defined

In [76]:
dsc

,Open_1_count,Open_1_mean,Open_1_std,Open_1_min,Open_1_25%,Open_1_50%,Open_1_75%,Open_1_max,High_1_count,High_1_mean,High_1_std,High_1_min,High_1_25%,High_1_50%,High_1_75%,High_1_max,Low_1_count,Low_1_mean,Low_1_std,Low_1_min,Low_1_25%,Low_1_50%,Low_1_75%,Low_1_max,Close_1_count,Close_1_mean,Close_1_std,Close_1_min,Close_1_25%,Close_1_50%,Close_1_75%,Close_1_max,Volume_1_count,Volume_1_mean,Volume_1_std,Volume_1_min,Volume_1_25%,Volume_1_50%,Volume_1_75%,Volume_1_max,Open_2_count,Open_2_mean,Open_2_std,Open_2_min,Open_2_25%,Open_2_50%,Open_2_75%,Open_2_max,High_2_count,High_2_mean,High_2_std,High_2_min,High_2_25%,High_2_50%,High_2_75%,High_2_max,Low_2_count,Low_2_mean,Low_2_std,Low_2_min,Low_2_25%,Low_2_50%,Low_2_75%,Low_2_max,Close_2_count,Close_2_mean,Close_2_std,Close_2_min,Close_2_25%,Close_2_50%,Close_2_75%,Close_2_max,Volume_2_count,Volume_2_mean,Volume_2_std,Volume_2_min,Volume_2_25%,Volume_2_50%,Volume_2_75%,Volume_2_max,Open_3_count,Open_3_mean,Open_3_std,Open_3_min,Open_3_25%,Open_3_50%,Open_3_75%,Open_3_max,High_3_count,High_3_mean,High_3_std,High_3_min,High_3_25%,High_3_50%,High_3_75%,High_3_max,Low_3_count,Low_3_mean,Low_3_std,Low_3_min,Low_3_25%,Low_3_50%,Low_3_75%,Low_3_max,Close_3_count,Close_3_mean,Close_3_std,Close_3_min,Close_3_25%,Close_3_50%,Close_3_75%,Close_3_max,Volume_3_count,Volume_3_mean,Volume_3_std,Volume_3_min,Volume_3_25%,Volume_3_50%,Volume_3_75%,Volume_3_max,Open_4_count,Open_4_mean,Open_4_std,Open_4_min,Open_4_25%,Open_4_50%,Open_4_75%,Open_4_max,High_4_count,High_4_mean,High_4_std,High_4_min,High_4_25%,High_4_50%,High_4_75%,High_4_max,Low_4_count,Low_4_mean,Low_4_std,Low_4_min,Low_4_25%,Low_4_50%,Low_4_75%,Low_4_max,Close_4_count,Close_4_mean,Close_4_std,Close_4_min,Close_4_25%,Close_4_50%,Close_4_75%,Close_4_max,Volume_4_count,Volume_4_mean,Volume_4_std,Volume_4_min,Volume_4_25%,Volume_4_50%,Volume_4_75%,Volume_4_max,Open_5_count,Open_5_mean,Open_5_std,Open_5_min,Open_5_25%,Open_5_50%,Open_5_75%,Open_5_max,High_5_count,High_5_mean,High_5_std,High_5_min,High_5_25%,High_5_50%,High_5_75%,High_5_max,Low_5_count,Low_5_mean,Low_5_std,Low_5_min,Low_5_25%,Low_5_50%,Low_5_75%,Low_5_max,Close_5_count,Close_5_mean,Close_5_std,Close_5_min,Close_5_25%,Close_5_50%,Close_5_75%,Close_5_max,Volume_5_count,Volume_5_mean,Volume_5_std,Volume_5_min,Volume_5_25%,Volume_5_50%,Volume_5_75%,Volume_5_max,Open_6_count,Open_6_mean,Open_6_std,Open_6_min,Open_6_25%,Open_6_50%,Open_6_75%,Open_6_max,High_6_count,High_6_mean,High_6_std,High_6_min,High_6_25%,High_6_50%,High_6_75%,High_6_max,Low_6_count,Low_6_mean,Low_6_std,Low_6_min,Low_6_25%,Low_6_50%,Low_6_75%,Low_6_max,Close_6_count,Close_6_mean,Close_6_std,Close_6_min,Close_6_25%,Close_6_50%,Close_6_75%,Close_6_max,Volume_6_count,Volume_6_mean,Volume_6_std,Volume_6_min,Volume_6_25%,Volume_6_50%,Volume_6_75%,Volume_6_max,Open_7_count,Open_7_mean,Open_7_std,Open_7_min,Open_7_25%,Open_7_50%,Open_7_75%,Open_7_max,High_7_count,High_7_mean,...,Close_24_75%,Close_24_max,Volume_24_count,Volume_24_mean,Volume_24_std,Volume_24_min,Volume_24_25%,Volume_24_50%,Volume_24_75%,Volume_24_max,Open_25_count,Open_25_mean,Open_25_std,Open_25_min,Open_25_25%,Open_25_50%,Open_25_75%,Open_25_max,High_25_count,High_25_mean,High_25_std,High_25_min,High_25_25%,High_25_50%,High_25_75%,High_25_max,Low_25_count,Low_25_mean,Low_25_std,Low_25_min,Low_25_25%,Low_25_50%,Low_25_75%,Low_25_max,Close_25_count,Close_25_mean,Close_25_std,Close_25_min,Close_25_25%,Close_25_50%,Close_25_75%,Close_25_max,Volume_25_count,Volume_25_mean,Volume_25_std,Volume_25_min,Volume_25_25%,Volume_25_50%,Volume_25_75%,Volume_25_max,Open_26_count,Open_26_mean,Open_26_std,Open_26_min,Open_26_25%,Open_26_50%,Open_26_75%,Open_26_max,High_26_count,High_26_mean,High_26_std,High_26_min,High_26_25%,High_26_50%,High_26_75%,High_26_max,Low_26_count,Low_26_mean,Low_26_std,Low_26_min,Low_26_25%,Low_26_50%,Low_26_75%,Low_26_max,Close_26_count,Close_26_mean,Close_26_std,Close_26_min,Close_26_25%,Close_26_50%,Close_26_75%,Close_26_max,Volum

In [253]:
idcs = list(dfc[dfc.label==0].index )

In [280]:
pd.DataFrame([[target_list_real[i][0,0]/target_list_real[i].min(),target_list_real[i].max()/target_list_real[i][0,0]] for i in idcs]).describe()

,0,1
count,9497.000000,9497.000000
mean,1.001192,1.005122
std,0.001389,0.005510
min,1.000000,1.000000
25%,1.000355,1.002174
50%,1.000836,1.003585
75%,1.001563,1.006035
max,1.024757,1.091107


In [91]:
target_list_real[0].shape

(30, 4)

In [42]:
target_1_label = np.concatenate([np.array([(target_list_real[i].max()/target_list_real[i][0,3]),(target_list_real[i].min()/target_list_real[i][0,3])]) for i in idcs]).reshape((len(idcs),2)).mean(axis=0)

In [43]:
target_1_label

array([1.00139585, 0.9963694 ])

In [41]:
np.concatenate(target_1_label).reshape((len(target_1_label),2)).mean(axis=0)

array([1.00139585, 0.9963694 ])

In [79]:
label_i = 0
ta = dsc.loc[label_i]
idcs = list(dfc[dfc.label==label_i].index )
target_1_label = np.concatenate([np.array([(target_list_real[i].max()/target_list_real[i][0,3]),(target_list_real[i].min()/target_list_real[i][0,3])]) for i in idcs]).reshape((len(idcs),2)).mean(axis=0)
to_plot = pd.DataFrame(np.array([ta[[j+"_"+str(i)+"_mean" for i in range(1,31)]].to_list() for j in ["Open","High","Low","Close"]]).transpose(),columns=["Open","High","Low","Close"])
to_plot = to_plot
to_plot["time"]=range(1,31)
fig1 = go.Figure(data=go.Ohlc(x=to_plot['time'],
                             open=to_plot['Open'],
                             high=to_plot['High'],
                             low=to_plot['Low'],
                             close=to_plot['Close']))
  
# show the figure
fig1.update(layout_xaxis_rangeslider_visible=False)
fig1.show()
print(target_1_label)
for i in range(10):
    print(idcs[i*len(idcs)//10])
    real_vals = target_list_real[idcs[i*len(idcs)//10]]
    print(real_vals.min()/real_vals[0,0],real_vals.max()/real_vals[0,0])
    to_plot_real = pd.DataFrame(real_vals,columns=["Open","High","Low","Close"])
    to_plot_real["time"]=range(1,31)
    fig = go.Figure(data=go.Ohlc(x=to_plot_real['time'],
                                 open=to_plot_real['Open'],
                                 high=to_plot_real['High'],
                                 low=to_plot_real['Low'],
                                 close=to_plot_real['Close']))

    # show the figure
    fig.update(layout_xaxis_rangeslider_visible=False)
    fig.show()

    

[1.00203938 0.99433859]
0
0.9947670298944788 1.000681052147768


15855
0.9950104966331229 1.0000899217335066


31885
0.8411988248643116 1.0015881071013497


47364
0.9866348004862577 1.0003202688881159


63348
0.9945125654827929 1.0006831850746798


79352
0.9889831623764881 1.0046706642102827


94941
0.9998294484248569 1.0072315759375372


110969
0.9960351969945918 1.000343840091275


126531
0.9990618801251507 1.002285229340913


142312
0.9931632015868351 1.0065796723479283


In [24]:
cluster_pos_dict={}
for i in range(20):
    idcs = list(dfc[dfc.label==i].index )
    cluster_pos_dict["cluster_"+str(i)]= np.concatenate([np.array([(target_list_real[i].max()/target_list_real[i][0,3]),(target_list_real[i].min()/target_list_real[i][0,3])]) for i in idcs]).reshape((len(idcs),2)).mean(axis=0)

In [62]:
i

9

In [98]:
pd.DataFrame(np.concatenate([np.array([(target_list_real[i].max()/target_list_real[i][0,3]),(target_list_real[i].min()/target_list_real[i][0,3])]) for i in idcs]).reshape((len(idcs),2)),columns=["High","Low"]).describe()

,High,Low
count,7607.000000,7607.000000
mean,1.001750,0.991656
std,0.001885,0.008860
min,1.000000,0.829987
25%,1.000643,0.989994
50%,1.001274,0.993915
75%,1.002268,0.996250
max,1.039735,0.999733


In [28]:
cluster_dict = pd.DataFrame(cluster_pos_dict).transpose()

In [31]:
cluster_dict

,High,Low
cluster_0,1.003994,0.997763
cluster_1,1.004661,0.998300
cluster_2,1.001611,0.993993
cluster_3,1.001439,0.992767
cluster_4,1.007192,0.998425
cluster_5,1.002065,0.996056
cluster_6,1.003503,0.997469
cluster_7,1.001530,0.994353
cluster_8,1.006848,0.998548
cluster_9,1.007723,0.998224


In [30]:
cluster_dict.columns=["High","Low"]

In [32]:
cluster_dict.to_parquet("cluster_dict.parquet")

In [ ]:
cluster_pos_dict={
    "cluster_0":{
        "bounds":[1.00139585,0.9963694]
        "description":"logarithmically converging to low"
    },
    "cluster_1":{
        "bounds":[1.00099859,0.99629907]
        "description":"Genertally starts on top zigzags to zero, kind of volatile might want to increase stop/loss"
    },
    "cluster_2":{
        "bounds":[1.00508215,0.99877247]
        "description":"starts from mid to bottom exponents to top"
    },
}

(1.0044865403788634, 1.0040383644623927)

In [22]:
import plotly.graph_objects as go
  
# import python pandas package
import pandas as pd
  
# read the stock price dataset
  
# use go.OHLC function and pass the date, open,
# high, low and close price of the function


In [ ]:
from matplotlib.finance import candlestick2_ohlc
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import datetime as datetime
import numpy as np

quotes = np.array(...)

fig, ax = plt.subplots()
candlestick2_ohlc(ax,quotes['open'],quotes['high'],quotes['low'],quotes['close'],width=0.6)

xdate = [datetime.datetime.fromtimestamp(i) for i in quotes['time']]

ax.xaxis.set_major_locator(ticker.MaxNLocator(6))

def mydate(x,pos):
    try:
        return xdate[int(x)]
    except IndexError:
        return ''

ax.xaxis.set_major_formatter(ticker.FuncFormatter(mydate))

fig.autofmt_xdate()
fig.tight_layout()

plt.show()